In [1]:
from PIL import Image, ImageDraw

import toyplot.font
import toyplot.text

import IPython.display

import xml.etree.ElementTree as xml
import sys
import logging
logging.basicConfig(level=logging.DEBUG)
toyplot.log.setLevel(logging.DEBUG)

In [2]:
text = "This is very<br/>special text!"
style = {"font-family":"helvetica", "font-size":"16px"}
fonts = toyplot.font.ReportlabLibrary()

layout = toyplot.text.layout(text=text, style=style, fonts=fonts)

DEBUG:toyplot:build formatting model: <Element 'body' at 0x10fa55f50>
DEBUG:toyplot:build formatting model: <Element 'br' at 0x10fa55f90>
DEBUG:toyplot:compute size: <toyplot.text.Page object at 0x1046380d0>
DEBUG:toyplot:compute size: <toyplot.text.Line object at 0x1020e72d0>
DEBUG:toyplot:compute size: <toyplot.text.TextBox object at 0x1020e7350>
DEBUG:toyplot:compute size: <toyplot.text.Newline object at 0x102011d10>
DEBUG:toyplot:compute size: <toyplot.text.TextBox object at 0x10fa55dd0>


In [3]:
toyplot.text.dump(layout, size=True)

toyplot.text.Page 165.376x19.2
  toyplot.text.Line 165.376x19.2
    toyplot.text.TextBox 'This is very' 80.896x19.2
    toyplot.text.Newline 0x0
    toyplot.text.TextBox 'special text!' 84.48x19.2


In [4]:
def draw(box, image):
    if isinstance(box, toyplot.text.TextBox):
        print box
        xml.SubElement(
            image,
            "rect",
            x=str(box.left),
            y=str(box.top),
            width=str(box.content_width),
            height=str(box.content_height),
            fill="lightgray",
            stroke="gray",
        )
        
        baseline_offset = (box.style["line-height"] - box.text_height) * 0.5
        #print baseline_offset
        #baseline_offset=0
        
        style = toyplot.style.combine(
            box.style,
            {"white-space":"pre"},
        )
        
        xml.SubElement(
            image,
            "text",
            x=str(box.left),
            y=str(box.bottom - baseline_offset),
            fill="black",
            stroke="none",
            style=toyplot.style.to_css(style),
        ).text = box.text
        
    for child in box.children:
        draw(child, image)

image = xml.Element(
    "svg",
    width=str(int(layout.content_width+10)),
    height=str(int(layout.content_height+10)),
    style="border: 1px solid #f0f0f0",
)
draw(layout, image)
IPython.display.SVG(xml.tostring(image))

In [5]:
reference = "<span style='font-family: helvetica; font-size: 16px'><span style='background: lightgray; vertical-align: baseline'>This is </span><span style='background: lightgray; vertical-align: baseline; font-size: 120%; font-weight:bold'>very<br/>special</span><span style='background: lightgray; vertical-align: baseline'> text!</span><span>"
IPython.display.HTML(reference)